In [1]:
# Imports
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

import datetime


#Settings
pd.set_option("display.max_rows", 70)
pd.set_option("display.max_columns", 101)

# Full_df read

In [2]:
# read

full_df = pd.read_pickle('../data/processed/full_df.pkl')

full_df

,leg_no,fn_carrier,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration_x,change_reason_code,dep_delay,Ac Type Code,trans_time,sched_trans_time,Sched Groundtime,Act Groundtime,day_of_origin,ac_type,fn_number_y,ac_registration_y,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration,flt_event_number,leg,rot_mismatch,actual_block_time,block_delay,ground_delay,day_of_week,hour_of_day_dep,hour_of_day_arr,minute_of_day_dep,Crewchange,act_groundtime_before,sched_groundtime_before,Crewchange_before,act_groundtime_new,sched_groundtime_new,rows_to_drop,rows_to_drop_grounddelay
9,272024970,East Carmen Airlines,2019-06-01,2019-06-01 03:25:00,2019-06-01,2019-06-01 06:45:00,2019-06-01 03:50:00,2019-06-01 07:01:00,ECLBAX,other problem,25.0,320,0,0,95.0,94.0,2019-06-01,321,EC3292,ECLBAX,45.0,New Jessica,East Carmen,South Nathaniel,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 08:20:00,2019-06-01 10:35:00,95.0,272024970.0,272022230.0,27.0,NaN,25.0,1,New Jessica-East Carmen,False,191.0,-9.0,25.0,5,3,6,205,both,94.0,95.0,first flt of day,94.0,95.0,0,0
120,272022230,East Carmen Airlines,2019-06-01,2019-06-01 08:20:00,2019-06-01,2019-06-01 10:35:00,2019-06-01 08:35:00,2019-06-01 10:41:00,ECLBAX,other problem,15.0,320,60,60,75.0,120.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2,East Carmen-South Nathaniel,False,126.0,-9.0,15.0,5,8,10,500,no change,94.0,95.0,both,120.0,75.0,0,0
198,272212848,East Carmen Airlines,2019-06-01,2019-06-01 11:50:00,2019-06-01,2019-06-01 14:15:00,2019-06-01 12:41:00,2019-06-01 14:52:00,ECLBAX,rotational problem,51.0,320,68,75,80.0,86.0,2019-06-01,321,EC3258,ECLBAX,45.0,South Nathaniel,East Carmen,Joneshaven,2019-06-01 11:50:00,2019-06-01 14:15:00,2019-06-01 15:35:00,2019-06-01 17:00:00,80.0,272212848.0,271997824.0,25.0,NaN,NaN,3,South Nathaniel-East Carmen,False,131.0,-14.0,51.0,5,11,14,710,both,120.0,75.0,no change,86.0,80.0,0,0
283,271997824,East Carmen Airlines,2019-06-01,2019-06-01 15:35:00,2019-06-01,2019-06-01 17:00:00,2019-06-01 16:18:00,2019-06-01 17:32:00,ECLBAX,rotational problem,43.0,320,64,70,50.0,32.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,East Carmen-Joneshaven,False,74.0,-11.0,43.0,5,15,17,935,no change,86.0,80.0,both,32.0,50.0,0,0
317,271998033,East Carmen Airlines,2019-06-01,2019-06-01 17:50:00,2019-06-01,2019-06-01 19:10:00,2019-06-01 18:04:00,2019-06-01 19:14:00,ECLBAX,other problem,14.0,320,42,50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,5,Joneshaven-East Carmen,False,70.0,-10.0,14.0,5,17,19,1070,last flt of day,32.0,50.0,no change,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11856,272211046,East Carmen Airlines,2019-06-30,2019-06-30 10:30:00,2019-06-30,2019-06-30 12:10:00,2019-06-30 10:34:00,2019-06-30 12:05:00,ECLXEX,other problem,4.0,320,37,45,50.0,85.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,3,Lisamouth-East Carmen,False,91.0,-9.0,4.0,6,10,12,630,both,39.0,45.0,no change,85.0,50.0,0,0
11916,271969272,East Carmen Airlines,2019-06-30,2019-06-30 13:00:00,2019-06-30,2019-06-30 14:15:00,2019-06-30 13:30:00,2019-06-30 14:36:00,ECLXEX,rotational problem,30.0,320,0,0,45.0,54.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,East Carmen-West Jason,False,66.0,-9.0,30.0,6,13,14,780,no change,85.0,50.0,both,54.0,45.0,0,0
11959,271969452,East Carmen Airlines,2019-06-30,2019-06-30 15:00:00,2019-06-30,2019-06-30 16:20:00,2019-06-30 15:30:00,2019-06-30 16:40:00,ECLXEX,rotational problem,30.0,320,41,45,45.0,57.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT

# Drop outliers

Generally, we assume that the many unplausible values originate from mistakes arising from data handling and manipulation and not from e.g., scheduling mistakes for negative scheduled groundtime. Consequently, we delete outliers and some negative values *before* we do the train/test split. This might be dangerous if those mistakes are not wrong data. If this would be assumed, one would need to delete them after the train/test split

In [3]:
# Function to identify outliers 
def find_outliers(df, col):
    q25 = df[col].quantile(0.25)
    q75 = df[col].quantile(0.75)
    iqr = q75 - q25
    limit_lower = q25 - 1.5 * iqr
    limit_upper = q75 + 1.5 * iqr

    outliers = df.index[(df[col] < limit_lower) | (df[col] > limit_upper)]

    return outliers

outlier_idx = []
full_df_columns = full_df[['Sched Groundtime', 'Act Groundtime', 'block_delay', 'ground_delay']].columns
for col in full_df_columns:
    if full_df[col].dtype in ['int64', 'float64']:
        outlier_idx.extend(find_outliers(full_df, col))

full_df = full_df.drop(outlier_idx)

# Drop negative ground times
print(full_df.shape)
full_df=full_df[full_df['Sched Groundtime']>0]
print(full_df.shape)
full_df=full_df[full_df['Act Groundtime']>0]
print(full_df.shape)

full_df.shape

(9814, 52)
(7931, 52)
(7499, 52)


(7499, 52)

# Drop columns

In [4]:
adv_drop_columns = [
    #'leg_no',
    'fn_number_y',
    'fn_carrier',
    'dep_sched_date',
    'dep_sched_time',
    'arr_sched_date',
    'arr_sched_time',
    'm_offblockdt',
    'm_onblockdt',
    'change_reason_code',
    'dep_delay', # we recalculated this and called it ground_delay
    'day_of_origin',
    'ac_type',
    'ac_registration_y',
    'dep_leg_inbound', # Same as dep_ap_sched
    'arr_leg_inbound', # This is always East Carmen
    'sched_inbound_dep',
    'sched_inbound_arr',
    'sched_outbound_dep',
    'sched_outbound_arr',
    'leg_inbound',
    'leg_outbound',
    'catering_duration', # little observations
    'cleaning_duration', # little observations
    'pax_boarding_duration',
    'rot_mismatch',
    'rows_to_drop',
    'arr_leg_outbound',
    'mingt', # Todo: Maybe impute as dependent on AC type
    'sched_turnaround', # Todo: Maybe impute
    #'trans_time',
    'actual_block_time', # we drop this as we do not know actuals at the beginning of the day and would need another forecast for act block time
    #'Sched Groundtime',
    'Act Groundtime', # we drop this as it is an actual value
    #'Crewchange'
    'act_groundtime_new'
    'sched_groundtime_new'
]

full_df = full_df.drop(adv_drop_columns, axis = 1)

In [5]:
full_df.describe()

,leg_no,trans_time,sched_trans_time,Sched Groundtime,flt_event_number,block_delay,ground_delay,day_of_week,hour_of_day_dep,hour_of_day_arr,minute_of_day_dep,act_groundtime_before,sched_groundtime_before,act_groundtime_new,sched_groundtime_new,rows_to_drop_grounddelay
count,7.499000e+03,7499.000000,7499.000000,7499.000000,7499.000000,7499.000000,7499.000000,7499.000000,7499.000000,7499.000000,7499.000000,7465.000000,7499.000000,7499.000000,7499.000000,7499.0
mean,2.720534e+08,27.121216,32.653687,53.408454,3.114415,-5.266969,21.120149,3.078544,9.577410,11.205627,600.522070,65.684528,59.509935,63.961061,53.408454,0.0
std,9.788998e+04,30.303605,30.511317,14.125292,1.731967,6.827403,17.686697,2.019138,4.053355,4.089048,245.048642,51.173695,38.269750,32.351115,14.125292,0.0
min,2.719556e+08,-129.000000,0.000000,15.000000,1.000000,-25.000000,0.000000,0.000000,0.000000,3.000000,25.000000,-73.000000,-55.000000,-20.000000,15.000000,0.0
25%,2.719818e+08,0.000000,0.000000,45.000000,2.000000,-10.000000,10.000000,1.000000,6.000000,7.000000,395.000000,40.000000,45.000000,40.000000,45.000000,0.0
50%,2.720115e+08,26.000000,40.000000,50.000000,3.000000,-5.000000,16.000000,3.000000,10.000000,11.000000,605.000000,61.000000,50.000000,62.000000,50.000000,0.0
75%,2.721049e+08,48.000000,50.000000,60.000000,4.000000,-1.000000,29.000000,5.000000,13.000000,14.000000,790.000000,85.000000,65.000000,84.000000,60.000000,0.0
max,2.723589e+08,280.000000,310.000000,95.000000,14.000000,15.000000,70.000000,6.000000,19.000000,20.000000,1160.000000,701.000000,680.000000,166.000000,95.000000,0.0


# Drop all NA

In [6]:
print(full_df.shape)
full_df = full_df.dropna(how = 'any')
print(full_df.shape)

(7499, 21)
(7465, 21)


# Save full_df

In [7]:
full_df.to_pickle('../data/finalized/full_df.pkl')
full_df.shape

(7465, 21)